In [4]:
import plotly.express as px
import streamlit as st
import plotly.io as pio
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime


from mainnet_launch.constants import *

from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_df,
    get_highest_value_in_field_where,
    _exec_sql_and_cache,
)
import plotly.express as px
import plotly.io as pio
import pandas as pd

pio.templates.default = "plotly_white"

rebalance_events = get_full_table_as_df(RebalanceEvents)
rebalance_plans = get_full_table_as_df(RebalancePlans)

In [5]:
df = pd.merge(rebalance_events, rebalance_plans, left_on="rebalance_file_path", right_on="file_name", how="left")
df.columns

Index(['tx_hash', 'autopool_vault_address_x', 'chain_id_x',
       'rebalance_file_path', 'destination_out_x', 'destination_in_x',
       'quantity_out', 'quantity_in', 'safe_value_out', 'safe_value_in',
       'spot_value_in', 'spot_value_out', 'spot_value_in_solver_change',
       'file_name', 'datetime_generated', 'autopool_vault_address_y',
       'chain_id_y', 'solver_address', 'rebalance_type', 'destination_out_y',
       'token_out', 'destination_in_y', 'token_in', 'move_name', 'amount_out',
       'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value',
       'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value',
       'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank',
       'projected_swap_cost', 'projected_net_gain', 'projected_gross_gain',
       'projected_slippage'],
      dtype='object')

In [ ]:
rebalance_hash = "0x1f9caa5ccfe6fadbb3f38aef1d7f36ba764da482b979f2c658e1458916e8d66e"
# min_amount_in from the plan
# quantity_in
local_df = df[df["tx_hash"] == rebalance_hash].copy()
local_df.T

# I suspect that for balancer pools
# the min amount out is not right,

# 99130288256157116100 the amount of lp tokens created here

,3297
tx_hash,0x1f9caa5ccfe6fadbb3f38aef1d7f36ba764da482b979...
autopool_vault_address_x,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
chain_id_x,1
rebalance_file_path,rebalance_plan_1755172103_0x0A2b94F6871c1D7A32...
destination_out_x,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
destination_in_x,0x9b163e15121816BE53F8d5c85fbefD6e6D9bebCd
quantity_out,100.0
quantity_in,99.036096
safe_value_out,100.0
safe_value_in,99.899147


In [ ]:
autopool_names = {a.autopool_eth_addr: a.name for a in ALL_AUTOPOOLS}
autopool_names

{'0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56': 'autoETH',
 '0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5': 'balETH',
 '0xE800e3760FC20aA98c5df6A9816147f190455AF3': 'autoLRT',
 '0xAADf01DD90aE0A6Bb9Eb908294658037096E0404': 'baseETH',
 '0x35911af1B570E26f668905595dEd133D01CD3E5a': 'dineroETH',
 '0xa7569A44f348d3D70d8ad5889e50F78E33d80D35': 'autoUSD',
 '0x9c6864105AEC23388C89600046213a44C384c831': 'baseUSD',
 '0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d': 'autoDOLA',
 '0xCb119265AA1195ea363D7A243aD56c73EA42Eb59': 'sonicUSD'}

In [ ]:
df.columns

Index(['tx_hash', 'autopool_vault_address_x', 'chain_id_x',
       'rebalance_file_path', 'destination_out_x', 'destination_in_x',
       'Actual Amount Out', 'Actual Amount In', 'safe_value_out',
       'safe_value_in', 'spot_value_in', 'spot_value_out',
       'spot_value_in_solver_change', 'file_name', 'datetime_generated',
       'autopool_vault_address_y', 'chain_id_y', 'solver_address',
       'rebalance_type', 'destination_out_y', 'token_out', 'destination_in_y',
       'token_in', 'move_name', 'Expected Amount Out', 'amount_out_safe_value',
       'Minimum Expected Amount In', 'min_amount_in_safe_value',
       'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value',
       'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank',
       'projected_swap_cost', 'projected_net_gain', 'projected_gross_gain',
       'projected_slippage'],
      dtype='object')

In [ ]:
rebalance_events.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'spot_value_in_solver_change'],
      dtype='object')

In [ ]:
df = df.rename(
    columns={
        "min_amount_in": "Minimum Expected Amount In",  # from the plan
        "amount_out": "Expected Amount Out",
        "quantity_out": "Actual Amount Out",  # from the event
        "quantity_in": "Actual Amount In",  # from the event
    }
)


local_df = df[
    [
        "move_name",
        "spot_value_in_solver_change",
        "Minimum Expected Amount In",
        "Expected Amount Out",
        "Actual Amount Out",
        "Actual Amount In",
        "datetime_generated",
        "autopool_vault_address_x",
        "file_name",
    ]
].copy()
local_df["autopool_name"] = local_df["autopool_vault_address_x"].map(autopool_names)
local_df["Excess Above"] = local_df["Actual Amount In"] - local_df["Minimum Expected Amount In"]

local_df["percentage_excess"] = (local_df["Excess Above"] / local_df["Minimum Expected Amount In"]) * 100
px.box(
    local_df,
    x="percentage_excess",
    y="autopool_name",
)

In [ ]:
px.scatter(
    local_df,
    y="spot_value_in_solver_change",
    x="datetime_generated",
    color="autopool_name",
)

In [ ]:
# the eth destinations are 1:1 with amount and and min amount out

In [ ]:
px.histogram(local_df, x="percentage_excess", color="autopool_name")

In [ ]:
local_df[local_df["percentage_excess"] < 0.01]

,move_name,Minimum Expected Amount In,Expected Amount Out,Actual Amount Out,Actual Amount In,datetime_generated,autopool_vault_address_x,file_name,autopool_name,Excess Above,percentage_excess
0,autoETH -> ethx-f,20.194820,20.879813,20.879813,20.194820,2025-01-24 07:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1737703821_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
1,autoETH -> osETH-rETH,59.339613,61.956502,61.956502,59.339613,2025-04-29 07:30:39+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1745911839_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
2,autoETH -> pxethweth,41.596185,41.638486,41.638486,41.596185,2025-05-20 10:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1747737024_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
3,autoETH -> pxETH/wETH,25.478596,25.547703,25.547703,25.478596,2025-01-13 13:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1736775021_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
4,autoETH -> ethx-f,96.908894,100.000000,100.000000,96.908894,2024-12-22 01:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1734831021_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3235,autoETH -> Surge Fluid wstETH-wETH,29.201766,29.482520,29.482520,29.201766,2025-08-09 23:48:23+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1754783303_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
3252,autoETH -> Surge Fluid wstETH-wETH,23.464670,23.688371,23.688371,23.464670,2025-08-10 23:48:22+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1754869702_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
3256,autoETH -> Surge Fluid wstETH-wETH,21.505724,21.709212,21.709212,21.505724,2025-08-11 11:48:23+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1754912903_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0
3272,autoETH -> Surge Fluid wstETH-wETH,97.464265,98.448302,98.448302,97.464265,2025-08-12 11:48:23+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,rebalance_plan_1754999303_0x0A2b94F6871c1D7A32...,autoETH,0.0,0.0


In [ ]:
# claim:
# where we get the where we don't get a premium, somthing is bad.

In [ ]:
rebalance_events["quantity_in"]

0           20.194820
1           59.339613
2           41.596185
3           25.478596
4           96.908894
            ...      
3289     13076.346868
3290     17011.744576
3291    610275.796585
3292     32278.297555
3293     23188.975178
Name: quantity_in, Length: 3294, dtype: float64

In [ ]:
df[["min_amount_in", "amount_out", "quantity_out", "quantity_in", "amount_in"]]

KeyError: "None of [Index(['min_amount_in', 'amount_out', 'quantity_out', 'quantity_in',\n       'amount_in'],\n      dtype='object')] are in the [columns]"

In [ ]:
rebalance_events

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,spot_value_in_solver_change
0,0x62a2725dbce1ada79e0f7eff3de740ea81232f030a5f...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1737703821_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573,20.879813,20.194820,20.879813,20.855487,20.862360,20.879813,0.003132
1,0x2c4950ee4d5b52f399284535b5fdf2e16aa3884930f4...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1745911839_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9,61.956502,59.339613,61.956502,61.935304,61.936455,61.956502,0.009293
2,0x464ccc26825b7a474de47fd45405f9cd366fe74a70f7...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1747737024_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,41.638486,41.596185,41.638486,41.604601,41.609905,41.638486,0.000000
3,0x8d648f557c3f8789847ed78c67db319a14537973ad54...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1736775021_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,25.547703,25.478596,25.547703,25.529665,25.527716,25.547703,0.000000
4,0xadf8e185ec280886d9edd70531a2722945ccd06ba5ed...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1734831021_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573,100.000000,96.908894,100.000000,99.916897,99.918070,100.000000,0.015000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0x9752d10aeee50286ca418e121ffc6e4a601220a6e95a...,0x9c6864105AEC23388C89600046213a44C384c831,8453,None,0x9c6864105AEC23388C89600046213a44C384c831,0xd3a721711b41c4f22C22A0727853fE322B84e931,13836.178000,13076.346868,13836.178000,13836.174156,13831.231297,13836.178000,0.000000
3290,0x675d73e93b4f5de71cd0106d21b45afca488d3ad2c0b...,0x9c6864105AEC23388C89600046213a44C384c831,8453,None,0x9c6864105AEC23388C89600046213a44C384c831,0xd3a721711b41c4f22C22A0727853fE322B84e931,18000.000000,17011.744576,18000.000000,17999.990842,17993.424308,18000.000000,0.000000
3291,0x2b88b5dc626e30e9205219aea40e54294602fcc22d98...,0x9c6864105AEC23388C89600046213a44C384c831,8453,None,0x4eeDC6d6126ae9C6D92D03bB355eEAe5E47d2b03,0xd3a721711b41c4f22C22A0727853fE322B84e931,612358.376382,610275.796585,645745.992137,645745.636158,645520.444389,645520.644255,0.000000
3292,0xb8e0015e60f343a7d49a0b0a2167df84037e5f26b2fb...,0x9c6864105AEC23388C89600046213a44C384c831,8453,None,0x9c6864105AEC23388C89600046213a44C384c831,0xd3a721711b41c4f22C22A0727853fE322B84e931,34151.983250,32278.297555,34151.983250,34151.955893,34139.851532,34151.983250,0.000000


In [ ]:
rebalance_plans.columns

Index(['file_name', 'datetime_generated', 'autopool_vault_address', 'chain_id',
       'solver_address', 'rebalance_type', 'destination_out', 'token_out',
       'destination_in', 'token_in', 'move_name', 'amount_out',
       'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value',
       'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value',
       'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank',
       'projected_swap_cost', 'projected_net_gain', 'projected_gross_gain',
       'projected_slippage'],
      dtype='object')